In [335]:
from abc import  ABC, abstractmethod

class Readable(ABC):
    def __init__(self): pass
    
    @abstractmethod
    def get(): pass

class Writable(ABC):
    def __init__(self): pass
    
    @abstractmethod
    def put(): pass

In [ ]:
import aiohttp
import asyncio
import os
from fake_useragent import UserAgent
from typing import Dict
class WorkerWithHtml(Readable):
    
    def __init__(self):
        super().__init__()

        # Объявление переменных, которые будут использоваться этим Worker-ом
        self._url = None
        self._user = None
        self.__header = None

    async def get(self, 
                  session: aiohttp.ClientSession, 
                  url: str, 
                  accept: str= '*/*',
                  header: Dict[str, str]= None):
        
        self._url = url
        self._user = UserAgent().random
        
        if not header:
            self.__header = {'Accept' : accept, 
                             'User-Agent': self._user}
        else:
            self.__header = header

        # Чтение данных из сайта
        data = None
        async with session.get(url= self._url, 
                               headers= self.__header) as response:
            data = await response.text()

        return data
    
    @property
    def user(self):
        return  self._user

    async def _read_main_site(self, path_main_site: str, url):

        # Забираем данные с основного сайта и кладем их в файла 
        data = None
        async with aiohttp.ClientSession() as session:
            data = await self.get(session, url= url)
        
        return data

In [337]:
import aiofiles
from typing import Any
import json

class WorkerWithFiles(Readable, Writable):

    def __init__(self):
        super().__init__()

    async def get(self, path: str):
        data = None
        async with aiofiles.open(file= path) as file:
            data = await file.read()
        return data
    
    async def put(self, path: str, data: Any):
        async with aiofiles.open(file= path, mode= 'w') as file:
            await file.write(data)
    
    async def _put_json_file(self, path: str, data: Any):
        async with aiofiles.open(path, 'w') as file:
            json_str = json.dumps(obj= data, indent= 4, ensure_ascii= False) 
            await file.write(json_str)

In [338]:
from bs4 import BeautifulSoup

class Parser(ABC):
    def __init__(self, base_url: str):
        self.__file_worker = WorkerWithFiles()
        self.__html_worker = WorkerWithHtml()
        self.base_url = base_url


    @abstractmethod
    def parsing(): pass

    @property
    def user_agent(self):
        return self.__html_worker.user
    
    async def put_file(self, path: str, data: Any):
        await self.__file_worker.put(path= path, data= data)
    

    async def get_file(self, path: str):
        return await self.__file_worker.get(path= path)

    
    async def get_html(self, 
                       session: aiohttp.ClientSession,
                       url: str, 
                       accept: str= '*/*',
                       with_delay= True, 
                       header: Dict[str, str]= None):
        return await self.__html_worker.get(session= session, url= url, accept= accept, with_delay= with_delay, header= header)
    

    async def _read_main_site_and_save(self, path_main_site: str, file_name_for_main_site: str):
        
        data = await self.__html_worker._read_main_site(file_name_for_main_site, self.base_url)
        
        # сохраняет данные в файл
        if BeautifulSoup(data, 'lxml').find(name= 'title').text == "Слишком много запросов":
            raise Exception("Вас заблокировали!")
        
        path= os.path.join(path_main_site, file_name_for_main_site)
        await self.put_file(path= path, data= data)
        

    async def _save_data_in_json_file(self, path: str, data: Any):
        await self.__file_worker._put_json_file(path, data)

In [ ]:
class ParserSite_23MET(Parser):
    def __init__(self, base_url):
        super().__init__(base_url)

    async def __get_hrefs_for_next_sites(self,
                                         file_path: str):

        #сначало парсим основной сайт
        html_file = await self.get_file(path= file_path)
        soup = BeautifulSoup(markup= html_file, 
                             features= 'lxml')
        
        items_html_with_href= soup.find(name= 'nav', 
                                        attrs={"id" : "left-container", "class" : "left-container-mainpage-static"})\
                                  .find(name= 'ul')\
                                  .find_all(name= 'a')
        
        # формируем json с ссылками на сайты, внутри которых будут еще одни сайты с сылками на данные
        hrefs_next_sites = dict()
        for item in items_html_with_href:
            item: BeautifulSoup
            item_name = item.get_text()
            item_href = self.base_url + item.get('href').replace('/price', '')
            hrefs_next_sites[item_name] = item_href

        return hrefs_next_sites

    # async def __sub_get_hrefs_for_next_sites(self, 
    #                                          file_path: str):
    #     html_file = await self.get_file(path= file_path)

    async def parsing(self,
                      file_name_for_main_site= 'main_site.html',
                      accept= '*/*',
                      dir_name= None,
                      with_save_data_in_file= True): 

#Дальше реализовать два способа реализации parsing-а c сохранением всех данных в файлы и брать записи из этих файлов... 
        
        path = os.getcwd()

        # создаю директорию, если есть dir_name
        if dir_name:
            path = os.path.join(path, dir_name)
            if not os.path.isdir(path):
                os.mkdir(path= path)
                print(f"Создал папку {dir_name} по пути {path}")
            else:
                print(f"Не стал создавать папку {dir_name}, т.к она уже существует!")
        
        try:
            await self._read_main_site_and_save(path_main_site= path, 
                                                file_name_for_main_site= file_name_for_main_site)

        except Exception as ex:
            print(ex)
            return ex

        # Получаем json с ссылками на сайты
        hrefs_next_sites = await self.__get_hrefs_for_next_sites(os.path.join(path, file_name_for_main_site)) 
        
        # # Сохраним полученный json в файл
        # await self._save_data_in_json_file(path= os.path.join(path, file_name_for_main_site.split('.')[0] + '.json'),
        #                                    data= hrefs_next_sites)

        # Переходим по ссылкам
        header = {'Accept' : accept,
                  'User-agent' : self.user_agent}
        
        tasks = []
        async with aiohttp.ClientSession() as session:
            for submain_url in hrefs_next_sites.values():
                task =  asyncio.create_task(self.get_html(session= session,
                                                          url= submain_url,
                                                          header= header)) 
                tasks.append(task)
            html_codes = await asyncio.gather(*tasks)
        
        if with_save_data_in_file:
            # Сохраняем все полученные данные в файлы
            for i, detail_name in enumerate(hrefs_next_sites.keys()):
                await self.put_file(path= os.path.join(path, detail_name),
                                    data= html_codes[i])
        
        


In [340]:
worker = ParserSite_23MET("https://23met.ru/price")
await worker.parsing(dir_name= 'data')

Создал папку data по пути /home/ranil/Рабочий стол/Project/ПРОЕКТ/parsing/data
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на 2
Пауза на

FileNotFoundError: [Errno 2] No such file or directory: '/home/ranil/Рабочий стол/Project/ПРОЕКТ/parsing/data/Балка б/у'